In [1]:
import os
os.chdir("../")
%pwd

'd:\\PracticeProjects\\NL_Text_Summarization\\NL_TEXT_SUMMARIZER'

In [2]:
# This block will later be moved to entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelEvaluationConfig:
  root_dir: Path
  data_path: Path
  model_path: Path
  tokenizer_path: Path
  metric_file_name: Path

In [3]:
# Define Config Manager
from NLPTextSummarizer.constants import *
from NLPTextSummarizer.utils.common import read_yaml, create_directories

# 4. Update the configuration manager(./src/project_name/config/configuration.py)
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        #print(CONFIG_FILE_PATH)
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 
        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        # Move config path dict from config.yaml
        config = self.config.model_evaluation

        # create the directories as mentioned in config.yaml
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            metric_file_name = config.metric_file_name
        )
        return model_evaluation_config

In [4]:
# Define components
# 5. Update the components (./src/project_name/components/__init__.py)
import os
from NLPTextSummarizer.logging import logger
from transformers import DataCollatorForSeq2Seq # type: ignore
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments # type: ignore
from datasets import load_from_disk, load_metric # type: ignore
import torch # type: ignore
from tqdm import tqdm # type: ignore
import pandas as pd # type: ignore

class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config
    
    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        rouge_metric = load_metric("rouge", trust_remote_code=True)
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        score = self.evaluate_score( dataset_samsum_pt["test"], rouge_metric,model, tokenizer, batch_size=2, device = device,  column_text="dialogue", column_summaries="summary")
        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
        pd.DataFrame(rouge_dict, index=[f"pegasus"]).to_csv(self.config.metric_file_name, index = False)      

    def chunks(self, list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i:i+batch_size]

    def evaluate_score(self, dataset, metric, model, tokenizer, batch_size = 16, device = "cpu", column_text = "article", column_summaries = "highlights"):
        article_batches = list(self.chunks(dataset[column_text], batch_size))
        target_batches = list(self.chunks(dataset[column_summaries], batch_size))
        for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total = len(article_batches)):
            inputs = tokenizer(article_batch, max_length = 1024, truncation = True, padding = "max_length", return_tensors = "pt")
            summaries = model.generate(
                input_ids = inputs["input_ids"].to(device),
                attention_mask = inputs["attention_mask"].to(device),
                length_penalty = 0.8,
                max_length = 128
            )
            decoded_summaries = [tokenizer.decode(s, skip_special_tokes = True, clean_up_tokenization_spaces = True) for s in summaries]
            decoded_summaries = [d.replace("<n>"," ") for d in decoded_summaries]
            metric.add_batch(predictions = decoded_summaries, references = target_batch)

            
        score = metric.compute()
        return score


d:\Softwares\Anaconda3\envs\NLP_TextSummarization_VENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-08 02:08:31,126: INFO: config: PyTorch version 2.3.0+cu118 available.]


In [5]:
# Pipeline component
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_evaluation_config) 
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2024-05-08 02:08:31,447: INFO: common: yaml file: D:\PracticeProjects\NL_Text_Summarization\NL_TEXT_SUMMARIZER\config\config.yaml loaded successfully]
[2024-05-08 02:08:31,449: INFO: common: yaml file: D:\PracticeProjects\NL_Text_Summarization\NL_TEXT_SUMMARIZER\params.yaml loaded successfully]
[2024-05-08 02:08:31,450: INFO: common: created directory at: artifacts]
[2024-05-08 02:08:31,451: INFO: common: created directory at: artifacts/model_evaluation]


UnboundLocalError: cannot access local variable 'device' where it is not associated with a value